In [11]:
import gc
import json
import linecache
from collections import Counter

import numpy as np
import tqdm
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer, ngrams_iterator
from torchtext.datasets import TextClassificationDataset
from torchtext.vocab import Vocab

## read data

In [2]:
df = pd.read_csv('data/Reviews.csv')
df = df[['Score', 'Text']]
df = df.dropna()
df = df.drop_duplicates('Text')

In [3]:
df.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [4]:
df['Text'].describe()

count                                                393579
unique                                               393579
top       This is the best cat litter box next to the Li...
freq                                                      1
Name: Text, dtype: object

In [5]:
df['Score'].unique()

array([5, 1, 4, 2, 3])

## build dataset

In [6]:
labels, reviews = df['Score'].to_numpy(), df['Text'].to_numpy()

make splits

In [7]:
def split_data(arrays, ratios=(0.7, 0.2, 0.1)):
    data_len = arrays[0].shape[0]
    assert all(a.shape[0] == data_len for a in arrays[1:])
    sizes = [r / sum(ratios) for r in ratios]
    sizes = [int(s * data_len) for s in sizes[:-1]]
    sizes.append(data_len - sum(sizes))
    start = 0
    finish = 0
    splits = []
    for s in sizes:
        finish += s
        splits.append([a[start:finish] for a in arrays])
        start += s
    return splits

train, valid, test = split_data((labels, reviews), (0.7, 0.2, 0.1))

In [8]:
def data_merge(data):
    labels, text = data
    return [{'label': int(l), 'text': row} for l, row in zip(labels, text)]

train = data_merge(train)
valid = data_merge(valid)
test = data_merge(test)

In [9]:
del df

tokenize text in datasets, add bigrams

In [10]:
! python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/vladimir/miniconda3/envs/clearscale/lib/python3.7/site-packages/en_core_web_sm
-->
/home/vladimir/miniconda3/envs/clearscale/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [10]:
def data_tokenize(data, tokenizer, lower, ngrams, cache=True):
    tokenizer = get_tokenizer(tokenizer)
    for entry in tqdm.tqdm(data, 'lines', len(data)):
        if lower:
            entry['text'] = entry['text'].lower()
        entry['text'] = tokenizer(entry['text'])
        entry['text'] = list(ngrams_iterator(entry['text'], ngrams))
    return data

tokenizer = 'spacy'
lower = True
ngrams = 2

train = data_tokenize(train, tokenizer, lower, ngrams)

lines: 100%|██████████| 275505/275505 [01:24<00:00, 3243.43it/s]


In [11]:
valid = data_tokenize(valid, tokenizer, lower, ngrams)
test = data_tokenize(test, tokenizer, lower, ngrams)

lines: 100%|██████████| 39359/39359 [00:12<00:00, 3069.20it/s]


save data

In [13]:
def save_tokenized(data, filename):
    with open(filename, 'wt') as f:
        f.writelines(json.dumps(l) + '\n' for l in tqdm.tqdm(data))

save_tokenized(train, 'train_tokenized.json')

100%|██████████| 275505/275505 [00:06<00:00, 45648.26it/s]


In [14]:
save_tokenized(valid, 'valid_tokenized.json')
save_tokenized(test, 'test_tokenized.json')

100%|██████████| 39359/39359 [00:00<00:00, 42899.17it/s]


load data if available

In [ ]:
def load_tokenized(filename):
    with open(filename, 'rt') as f:
        return [json.loads(l) for l in tqdm.tqdm(f.readlines())]

if train is None:
    train = load_tokenized('train_tokenized.json')

create vocabulary

In [5]:
def build_vocab(data, max_size=30000):
    counter = Counter()
    for entry in tqdm.tqdm(data):
        counter.update(entry['text'])
    return Vocab(counter, max_size)

vocab = build_vocab(train)

100%|██████████| 275505/275505 [00:10<00:00, 26010.17it/s]


create torch datasets

In [10]:
class JsonDataset(Dataset):
    def __init__(self, filename, vocab):
        self.filename = filename
        self.vocab = vocab
        self.label_dict = {i + 1: i for i in range(5)}
        self.data_len = None

    def __getitem__(self, index):
        with open(self.filename, 'rt') as f:
            for i, l in enumerate(f):
                if i == index:
                    break
        line = json.loads(l)
        label = line['label']
        label = self.label_dict[label]
        text = line['text']
        text = [torch.tensor(vocab[token]) for token in text]
        return label, text

    def __len__(self):
        if not self.data_len:
            with open(self.filename) as f:
                for i, l in enumerate(f):
                    pass
            self.data_len = i + 1
        return self.data_len

train_dataset = JsonDataset('train_tokenized.json', vocab)

In [9]:
valid_dataset = JsonDataset('valid_tokenized.json', vocab)
test_dataset = JsonDataset('test_tokenized.json', vocab)